In [1]:
import os

# Familirisation with data

In [2]:
data_path = "../data/jena_climate_2009_2016.csv"

In [3]:
with open(data_path) as f:
    data = f.read()
    lines = data.split("\n")
    header = lines[0].split(",")
    lines = lines[1:]

In [4]:
print(header)

['"Date Time"', '"p (mbar)"', '"T (degC)"', '"Tpot (K)"', '"Tdew (degC)"', '"rh (%)"', '"VPmax (mbar)"', '"VPact (mbar)"', '"VPdef (mbar)"', '"sh (g/kg)"', '"H2OC (mmol/mol)"', '"rho (g/m**3)"', '"wv (m/s)"', '"max. wv (m/s)"', '"wd (deg)"']


In [5]:
print(lines[:5])

['01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.30,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.30', '01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.40,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.10', '01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.90,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.60', '01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.20,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.00', '01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.10,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.30']


In [6]:
print(len(lines))

420451


## data -> numpy array

In [7]:
import numpy as np

In [8]:
float_data = np.zeros((len(lines), len(header) - 1))

In [9]:
print(float_data.shape)

(420451, 14)


In [10]:
for i, line in enumerate(lines):
    float_data[i] = line.split(",")[1:]

In [11]:
print(float_data.shape)

(420451, 14)


## data preprocessing

In [12]:
train_max_index, val_max_index = 200000, 300000

### scaling

In [13]:
mean_0 = float_data[:train_max_index].mean(axis = 0)
float_data -= mean_0
std_0 = float_data[:train_max_index].std(axis = 0)
float_data /= std_0

## data selection

In [14]:
lookback = int(5*24*60/10) # 5 days, 1 measuremnt per minute
delay = int(1*24*60/10)    # target is 1 day in the future
step = int(1*60/10)        # observations are smapled per an hour
batch_size = 128

In [15]:
def climate_generator(data,
                      lookback,
                      delay,
                      min_index,
                      max_index,
                      batch_size,
                      step,
                      shuffle = False):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(i, max_index, batch_size)
        else:
            if (i + batch_size) >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows), lookback//step, data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]

        yield samples, targetss
                          

In [16]:
train_gen = climate_generator(data = float_data,
                  lookback = lookback,
                  delay = delay,
                  min_index = 0,
                  max_index = train_max_index,
                  batch_size = batch_size,
                  step = step,
                  shuffle = False)

val_gen = climate_generator(data = float_data,
                  lookback = lookback,
                  delay = delay,
                  min_index = train_max_index + 1,
                  max_index = val_max_index,
                  batch_size = batch_size,
                  step = step,
                  shuffle = False)

test_gen = climate_generator(data = float_data,
                  lookback = lookback,
                  delay = delay,
                  min_index = val_max_index + 1,
                  max_index = None,
                  batch_size = batch_size,
                  step = step,
                  shuffle = False)

In [17]:
val_steps = val_max_index - train_max_index - 1 - lookback
test_steps = len(float_data) - val_max_index - 1 - lookback

In [18]:
print("val_gen stapes {}".format(val_steps))
print("test_steps stapes {}".format(test_steps))

val_gen stapes 99279
test_steps stapes 119730
